In [1]:
#import libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
train = pd.read_csv('C:/Users/MANISH/Desktop/Analytics vidya data/AmExpert/train_amex/train.csv')
user_history = pd.read_csv('C:/Users/MANISH/Desktop/Analytics vidya data/AmExpert/train_amex/historical_user_logs.csv')
test = pd.read_csv('C:/Users/MANISH/Desktop/Analytics vidya data/AmExpert/test_LNMuIYp/test.csv')

In [3]:
train_ID = train['session_id']
test_ID = test['session_id']

y = train['is_click']
train.drop('is_click',axis = 1, inplace = True)

In [4]:
train.drop('session_id',axis = 1,inplace = True)
test.drop('session_id',axis = 1,inplace = True)

In [5]:
categorical_features_indices = np.where(train.dtypes != np.float)[0]
categorical_features_indices

array([ 0,  1,  2,  3,  4,  5,  8, 12], dtype=int64)

In [6]:
data = pd.concat([train,test],axis = 0)

In [7]:
data['product_category_2'] = data['product_category_2'].fillna(data['product_category_2'].mean())
data['user_group_id'] = data['user_group_id'].fillna(data['user_group_id'].mean())
data['gender'] = data['gender'].fillna(data['gender'].mode()[0])
data['age_level'] = data['age_level'].fillna(data['age_level'].mean())
data['user_depth'] = data['user_depth'].fillna(data['user_depth'].mean())
data['city_development_index'] = data['city_development_index'].fillna(data['city_development_index'].mean())
data['var_1'] = data['var_1'].fillna(data['var_1'].mean())

In [8]:
from scipy.stats import boxcox

In [9]:
data.describe()

,user_id,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,age_level,user_depth,city_development_index,var_1
count,5.921490e+05,592149.000000,592149.000000,592149.000000,592149.000000,592149.000000,592149.000000,592149.000000,592149.000000,592149.000000
mean,5.461209e+05,292109.671470,28837.971259,3.166571,150216.320395,3.489550,2.790496,2.877995,2.558358,0.414529
std,3.294247e+05,132675.251267,20884.388892,1.349075,39861.636794,2.372939,1.053179,0.392586,0.787478,0.492641
min,4.000000e+00,82320.000000,1734.000000,1.000000,18595.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,2.582460e+05,118601.000000,13787.000000,2.000000,150216.320395,2.000000,2.000000,3.000000,2.000000,0.000000
50%,5.313420e+05,359520.000000,13787.000000,3.000000,150216.320395,3.000000,3.000000,3.000000,2.558358,0.000000
75%,8.274290e+05,404347.000000,51181.000000,4.000000,150216.320395,4.000000,3.000000,3.000000,3.000000,1.000000
max,1.141729e+06,414149.000000,60305.000000,5.000000,450184.000000,12.000000,6.000000,3.000000,4.000000,1.000000


In [10]:
# Normalize the data 

for i in ['product_category_2','user_depth','city_development_index']:
    data[i] = boxcox(data[i])[0]

In [11]:
train2 = data.iloc[:len(train),:]
test2 = data.iloc[len(train):,:]

In [12]:
from catboost import CatBoostClassifier
y_test = 0
for i in range(5):
    model = CatBoostClassifier(iterations=300, 
                               learning_rate=0.1, 
                               random_seed=i,
                               logging_level='Silent',
                               loss_function='Logloss',
                               eval_metric='AUC',
                               one_hot_max_size=5)
    model.fit(train2,y,cat_features = categorical_features_indices)
    y_test += model.predict_proba(test2)[:,1]

# Average of the predictions
y_test/5

array([0.07758683, 0.02250977, 0.02511761, ..., 0.05175622, 0.05988244,
       0.07914261])

In [13]:
# Assign to the sample submission 
sample = pd.DataFrame()
sample['session_id'] = test_ID
sample['is_click'] = y_test
# Output to csv
sample.to_csv('C:/Users/MANISH/Desktop/Analytics vidya data/AmExpert/train_amex/submission_10.csv',index = False)